In [13]:
import mysql.connector
import random
import string
import datetime
import names
# import sys
# sys.path.append(r'C:\Users\amand\project')
# import sqlpassword


In [14]:
def randomstring(stringLength, i):
    random.seed( i )
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

def randomstrnum(stringLength, i):
    random.seed( i )
    strnum = random.choices(string.ascii_uppercase + string.digits + string.ascii_lowercase, k = stringLength)
    return ''.join(strnum)

def email_ge(len, i):
    account = randomstring(len, i) + "@gmail.com"
    return account

def name_ge(i): 
    random.seed(i)
    name = names.get_full_name()
    return name

def short_name_ge(i): 
    random.seed(i)
    name = names.get_last_name()
    return name

def randomselect(list, num, i):
    random.seed(i)
    choice = random.choice(list)
    return choice

def subject_ge(i):
    subject = ['Chinese', 'English', 'Math']
    choice = randomselect(subject, 1, i)
    return choice


def insertion(tablename, list):
    s_word = "insert into "
    m_word = ' values ("'
    comma = '", "'
    query = s_word + tablename + m_word 
    for i in range(len(list)):
        query += str(list[i])
        if i != len(list) - 1:
            query += comma
    query += '");'
    print(query)
    mycursor.execute(query)

def randomnumber(i, start, end):
    random.seed(i)    
    return random.randrange(start, end)  

s_nouns = ["Choose the correct answer.", "Which is the most suitable answer?", "according to the textbook which is the definition of this term?"]

s = ["The test", "The grade", "The dead line", "Syllabus", "The HW requirement"]
v1 = ["has been", "will be"]
v2 = ["changed", "issued", "added"]
p = ["after class", ". Please look up yours", "Please report if there is any error."]
def announce_maker(i ):
    random.seed(i)
    return random.choice(s) + " " + random.choice(v1) + " " +  random.choice(v2) + " " + random.choice(p)

def sing_sen_maker(i):
    random.seed(i)    
    return random.choice(s_nouns)

def date_ge(i):
    year = randomnumber(i, 2015, 2020)
    month = randomnumber(i, 1, 13)
    day = randomnumber(i, 1, 30)
    return datetime.date(year, month, day)


In [15]:
  
def t_table(stringlen, i):
    mail = email_ge(stringlen, i)
    name = name_ge(i)
    sub = subject_ge(i)
    pw = randomstrnum(stringlen, i)
    insertion('teacher_list', [mail, sub, name, pw])
    return mail

def c_table(stringlen,mail, i):
    class_name = short_name_ge( i )
    class_code = randomstrnum(stringlen, i )
    insertion('class_list', [class_name, class_code, mail])
    return class_code

def s_table(stringlen, i ):
    student_name  = name_ge(i )
    student_email = email_ge(stringlen, i )
    s_password = randomstrnum(stringlen, i )
    insertion('student_list', [student_name, student_email, s_password])
    return student_email
    
def cs_table(class_code, student_email):
    insertion('c_s_belonging', [class_code, student_email])
    
def drawing(class_code, i):
    ans_grade = randomnumber(i, 0, 101)
    drawing_name = short_name_ge( i )
    date =  date_ge(i)

    insertion('drawing', [ans_grade, drawing_name, date, class_code])
    
def announce(class_code, i):
    content = announce_maker(i)
    date =  date_ge(i)
    insertion('announcement', [content, date, class_code])
    
def group(class_code, student_email, i):
    if randomnumber(i, 0, 2) == 1:
        grouping_for = short_name_ge(i  )
        group_num = randomnumber(i, 0, 5) 
        insertion('studentgroup', [group_num, grouping_for, class_code, student_email])

def test_answer(test_paper_id, i, quiz):
    for j in range(quiz):
        content_order = j + 1
        answers = randomnumber(i + j, 0, 5) 
        quiz_content = sing_sen_maker(i + j)
        insertion('test_paper_answer', [content_order, answers, quiz_content, test_paper_id])    

def class_test_student(stringlen, test_paper_id, class_code, i, current_i, s_dev, s_num, quiz, t_num):
    test_name = short_name_ge(i)
    test_date =  date_ge(i)
    insertion('class_test_assign', [ test_date, test_name, class_code, test_paper_id])    
    for j in range(s_num):

        student_reply = ''.join(str(random.randrange(1, 5))  for k in range(quiz))
        s_email = email_ge(stringlen, s_dev + j + current_i * t_num)
        insertion('test_taking', [student_reply, test_date, test_name, s_email, test_paper_id])    

 
def test_paper(stringlen, teacher_email, class_code, i , quiz, current_i, s_dev, s_num, t_num):
    if randomnumber(i, 0, 2) == 1:
        test_paper_name = short_name_ge(i)
        test_paper_id = randomstrnum(stringlen, i) 
        insertion('test_paper', [test_paper_name, test_paper_id, teacher_email ])  
        
        test_answer(test_paper_id, i, quiz)
        class_test_student(stringlen, test_paper_id, class_code, i, current_i, s_dev, s_num, quiz, t_num)




In [25]:
mydb = mysql.connector.connect(
  host = "localhost",
  user = "root",
  passwd = sqlpassword.ps,
  database="test"
  
)

print(mydb)
mycursor = mydb.cursor()

In [26]:
stringlen = 7
gap = 100
class_dev = gap 
t_dev = class_dev + gap
d_dev = t_dev + gap
s_dev = d_dev + gap

t_num = 10
s_num = 5
d_range_s = 0
d_range_end = 5
p_num = 1
quiz = 10


for i in range(t_num):
    t_mail = t_table(stringlen, i)
    c_code = c_table(stringlen, t_mail, i + class_dev)
    for k in range(randomnumber(i, d_range_s, d_range_end)):
        drawing(c_code, i * t_num + k + d_dev)
        announce(c_code, i * t_num + k + d_dev)        
    
    for j in range(s_num):
        s_mail = s_table(stringlen, j + i * t_num + s_dev)
        cs_table(c_code, s_mail)
        group(c_code, s_mail, i )
        
    for h in range(p_num):
        test_paper(stringlen, t_mail, c_code,  i * t_num + h + t_dev, quiz, i, s_dev, s_num, t_num)


        

        
 




insert into teacher_list values ("mynbiqp@gmail.com", "English", "Genevieve Gallegos", "qk0Q5Zm");
insert into class_list values ("Campbell", "J2lhj0n", "mynbiqp@gmail.com");
insert into drawing values ("76", "Dietrich", "2019-10-20", "J2lhj0n");
insert into announcement values ("The HW requirement will be issued after class", "2019-10-20", "J2lhj0n");
insert into drawing values ("51", "Ware", "2018-07-13", "J2lhj0n");
insert into announcement values ("Syllabus will be added . Please look up yours", "2018-07-13", "J2lhj0n");
insert into drawing values ("71", "Kodadek", "2019-09-18", "J2lhj0n");
insert into announcement values ("The HW requirement will be issued after class", "2019-09-18", "J2lhj0n");
insert into student_list values ("April Wendling", "jrizcsp@gmail.com", "TPo9m2w");
insert into c_s_belonging values ("J2lhj0n", "jrizcsp@gmail.com");
insert into studentgroup values ("3", "Hartl", "J2lhj0n", "jrizcsp@gmail.com");
insert into student_list values ("Susan Losoya", "pdnizsz@g

In [27]:
mycursor.close()
mydb.commit()

In [23]:
# mycursor.execute('delete from teacher_list;')
# mycursor.execute('delete from class_list;')
# mycursor.execute('delete from drawing;')
# mycursor.execute('delete from announcement;')
# mycursor.execute('delete from student_list;')
# mycursor.execute('delete from c_s_belonging;')
# mycursor.execute('delete from studentgroup;')
# mycursor.execute('delete from test_paper;')
# mycursor.execute('delete from test_paper_answer;')
# mycursor.execute('delete from class_test_assign;')
# mycursor.execute('delete from test_taking;')




